## This notebook performs CWFS for a target XXXYYY
##### The script, slews and acquires the target, takes CWFS data and applies focus correction, takes sequence of data

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-28818**

In [ ]:
TARGET='HD 74000'

In [ ]:
import os
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
script_cwfs = LatissCWFSAlign(index=1, remotes=False)  # this essentially calls the init method
await script_cwfs.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
# script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

## Perform CWFS observations

In [ ]:
cwfs_configuration = yaml.safe_dump({"filter": 'RG610', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
print(cwfs_configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script_cwfs.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
cwfs_config_data = script_cwfs.cmd_configure.DataType()
cwfs_config_data.config = cwfs_configuration
await script_cwfs.do_configure(cwfs_config_data)

In [ ]:
# set group ID as before
time_for_groupID=astropy.time.Time.now().isot
group_id_data = script_cwfs.cmd_setGroupId.DataType(
                groupId=time_for_groupID
            )
await script_cwfs.do_setGroupId(group_id_data)

In [ ]:
# Run this script while taking new data
script_cwfs.intra_visit_id = None
script_cwfs.extra_visit_id = None
script_cwfs.short_timeout = 10

In [ ]:
# run acquisition script
# results = await script_cwfs.run()
result = script_cwfs.cmd_run.DataType()
await script_cwfs.do_run(result)
await script_cwfs.done_task

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 

In [ ]:
await atcs.take_object(exptime=30)

# Perform Data sequence

In [ ]:
# Have to restart remotes since they'll have been closed from the acquisition
await script.start_task

In [ ]:
seq_configuration = yaml.safe_dump({"object_name": TARGET,
                                "do_acquire": False,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [4, 4, 1, 2, 1, 2], 
                                "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
                                    # RG610 and Ronchi
                                "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
                                "doPointingModel": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq_configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq_config_data = script.cmd_configure.DataType()
seq_config_data.config = seq_configuration
await script.do_configure(seq_configuration)

In [ ]:
# set groupID - but same as before so omitting

In [ ]:
# run data sequencing script
run_data = script.cmd_run.DataType()
await script.do_run(run_data)
await script.done_task